# Linear Regression

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.datasets import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

### 1. Non Vectorized implementation with single variable

$$
\hat{y} = wx + b \\
error^{(i)} = \hat{y}^{(i)} - y^{(i)} \\
\mathcal{J}(w, b) = \frac{1}{n} \sum_{i=1}^n{\hat{y}^{(i)} - y^{(i)})}^2 \\
\mathcal{J}(w, b) = \frac{1}{n} \sum_{i=1}^n{(\hat{y}^{(i)} - y^{(i)})}^2
$$

In [ ]:
#Function to calculate the cost
def compute_cost(x, y, w, b):
    m = x.shape[0] 
    cost = 0
    
    for i in range(m):
        f_wb = w * x[i] + b
        cost = cost + (f_wb - y[i])**2
    total_cost = 1 / (2 * m) * cost

    return total_cost

$$
\frac{\partial{\mathcal{J}}}{\partial{b}} = \frac{\partial{\mathcal{J}}}{\partial{\hat{y}^{(i)}}} \frac{\partial{\hat{y}^{(i)}}}{\partial{b}} \hspace{4 mm} = \frac{1}{m} \sum_{i=1}^m{2(b + w x^{(i)} - y^{(i)})} \\

\frac{\partial{\mathcal{J}}}{\partial{w}} = \frac{\partial{\mathcal{J}}}{\partial{\hat{y}^{(i)}}} \frac{\partial{\hat{y}^{(i)}}}{\partial{w}} = \frac{1}{m} \sum_{i=1}^m{2x^{(i)}(b + w x^{(i)} - y^{(i)})}
$$

In [ ]:
def compute_gradient(x, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      x (ndarray (m,)): Data, m examples 
      y (ndarray (m,)): target values
      w,b (scalar)    : model parameters  
    Returns
      dj_dw (scalar): The gradient of the cost w.r.t. the parameters w
      dj_db (scalar): The gradient of the cost w.r.t. the parameter b     
     """
    
    # Number of training examples
    m = x.shape[0]    
    dj_dw = 0
    dj_db = 0
    
    for i in range(m):  
        f_wb = w * x[i] + b 
        dj_dw_i = (f_wb - y[i]) * x[i] 
        dj_db_i = f_wb - y[i] 
        dj_db += dj_db_i
        dj_dw += dj_dw_i 
    dj_dw = dj_dw / m 
    dj_db = dj_db / m 
        
    return dj_dw, dj_db

$$
b = b - \eta \frac{\partial{\mathcal{J}}}{\partial{b}} \\

w = w - \eta \frac{\partial{\mathcal{J}}}{\partial{w}}
$$

In [ ]:
import math, copy
def gradient_descent(x, y, w_in, b_in, alpha, num_iters, cost_function, gradient_function): 
    """
    Performs gradient descent to fit w,b. Updates w,b by taking 
    num_iters gradient steps with learning rate alpha
    
    Args:
      x (ndarray (m,))  : Data, m examples 
      y (ndarray (m,))  : target values
      w_in,b_in (scalar): initial values of model parameters  
      alpha (float):     Learning rate
      num_iters (int):   number of iterations to run gradient descent
      cost_function:     function to call to produce cost
      gradient_function: function to call to produce gradient
      
    Returns:
      w (scalar): Updated value of parameter after running gradient descent
      b (scalar): Updated value of parameter after running gradient descent
      J_history (List): History of cost values
      p_history (list): History of parameters [w,b] 
      """
    
    w = copy.deepcopy(w_in) # avoid modifying global w_in
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    p_history = []
    b = b_in
    w = w_in
    
    for i in range(num_iters):
        # Calculate the gradient and update the parameters using gradient_function
        dj_dw, dj_db = gradient_function(x, y, w , b)     

        # Update Parameters using equation (3) above
        b = b - alpha * dj_db                            
        w = w - alpha * dj_dw                            

        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion 
            J_history.append( cost_function(x, y, w , b))
            p_history.append([w,b])
        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0:
            # print(f"Iteration {i:4}: Cost {J_history[-1]:0.2e} ",
            #       f"dj_dw: {dj_dw: 0.3e}, dj_db: {dj_db: 0.3e}  ",
            #       f"w: {w: 0.3e}, b:{b: 0.5e}")
            print(f"Iteration {i:4}: Cost {J_history[-1]} ",
                  f"dj_dw: {dj_dw}, dj_db: {dj_db}  ",
                  f"w: {w}, b:{b}")
 
    return w, b, J_history, p_history #return w and J,w history for graphing

In [ ]:
true_b = 1
true_w = 2
N = 100

np.random.seed(42)

# Create an array of the given shape and populate it with random samples 
# from a uniform distribution over [0, 1)
X = np.random.rand(N, 1)
epsilon = (.1 * np.random.randn(N, 1))

y = true_b + true_w * X + epsilon

In [ ]:
df = pd.DataFrame({ "x": X.tolist(), "y": y.tolist()})
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# initialize parameters
w_init = 0
b_init = 0
# some gradient descent settings
iterations = 10000
tmp_alpha = 1.0e-2
# run gradient descent
w_final, b_final, J_hist, p_hist = gradient_descent(X_train ,y_train, w_init, b_init, tmp_alpha, 
                                                    iterations, compute_cost, compute_gradient)

#Compare the w and b obtained with gradient descent with actual values
print(f"(w,b) found by gradient descent: ({w_final},{b_final})")

### 2. Load dataset for multiple linear regression

In [ ]:
# Load the Diabetes dataset
data = load_diabetes()

In [ ]:
data.feature_names

In [ ]:
print(data.DESCR)

In [ ]:
X = data.data
y = data.target

# Convert the iris dataset to a pandas dataframe
df = pd.DataFrame(data.data, columns=["age","sex","bmi","bp", "tc", "ldl", "hdl","tch", "ltg", "glu"])

# Add the target variable to the dataframe
df['target'] = data.target

# Print the first 5 rows of the dataframe
df.head()

In [ ]:
df.shape

##### 2.1 EDA

In [ ]:
df.isnull().sum()

In [ ]:
#describing dataframe
df.describe()

**Observations**:
1. All features except target have the same standard deviation
2. Apart from target column and a couple of exceptions, the rest of the features have the same order of magnitude (mean, min, max and percentiles).

In [ ]:
#applying mask
mask = np.triu(np.ones_like(df.corr()))

#correlation matrix
dataplot = sns.heatmap(df.corr(), annot=True, fmt='.2f', mask=mask)

**Observations**:
1. No high correlation between features. i.e. no multicollinearity to worry about

In [ ]:
sns.pairplot(df[['bmi', 'ltg', 'tch', 'target']])

In [ ]:
sns.regplot(data=df, x='bmi', y='target',line_kws={"color": "red"})
plt.title("BMI v/s Target")
plt.xlabel("BMI")
plt.ylabel("Target")
plt.show()

In [ ]:
sns.regplot(data=df, x='ltg', y='target',line_kws={"color": "red"})
plt.title("LTG v/s Target")
plt.xlabel("LTG")
plt.ylabel("Target")
plt.show()

**Observation**:

1. Relationship between ltg/bmi with target is mostly linear. There is slight heteroskedasticity

### 3. Multiple Linear Regression with Vectorized implementation

$$
\mathcal{J}(w) = \frac{1}{m} (Xw - y)^T(Xw - y) \\
\nabla_w J = \frac{2}{m} X^T (Xw - y) \\
\textbf{w} = \textbf{w} - \eta \nabla_w J 
$$

In [ ]:
def vectorized_compute_cost(X, y, w):
    m = len(y)
    predictions = np.dot(X, w)
    error = predictions - y
    cost = (1 / (2 * m)) * np.sum(error**2)
    return cost

In [ ]:
def vectorized_compute_gradient(X, y, w):
    m = len(y)
    predictions = np.dot(X, w)
    error = predictions - y
    gradient = (1 / m) * np.dot(X.T, error)
    return gradient


In [ ]:
def gradient_descent(X, y, w, learning_rate, num_epochs):
    m = len(y)
    loss_history = []

    for epoch in range(num_epochs):
        cost = vectorized_compute_cost(X, y, w)

        gradient = vectorized_compute_gradient(X, y, w)
        
        w -= learning_rate * gradient

        loss_history.append(cost)

    return w, loss_history

Data matrix X should be augmented with 1s in the first column that correspond to bias (intercept) weight w_0
$$
X = \begin{bmatrix}
1 & x^{(1)}_1 & .. & x^{(1)}_n \\
1 & x^{(2)}_1 & .. & x^{(2)}_n\\
1 & ..  & .. & ..\\
1 & x^{(m)}_1 & .. & x^{(m)}_n\\
\end{bmatrix}
$$

In [ ]:
data = load_diabetes()
X = data.data
y = data.target

X.shape
#notice augmenting the data matrix with first column of 1s 
X = np.hstack((np.ones(X.shape[0]).reshape(-1,1), X)) 
X[0:2,:]

In [ ]:
# Set hyperparameters
num_epochs = 50000
learning_rate = 0.1

# Initialize weight params
w = np.zeros(X.shape[1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
w, loss_history = gradient_descent(X_train, y_train, w, learning_rate, num_epochs)

In [ ]:
# Plot the loss curve
plt.plot(range(num_epochs), loss_history)
plt.xlabel("Epoch/Iteration")
plt.ylabel("Loss")
plt.title("Epoch/Iteration versus Loss")
plt.show()

### 4. Make predictions with custom vectorized linear regression

R-squared is very low. This is expected because Linear Regression is not a good model for diabetes dataset

In [ ]:
y_train_pred_custom = np.dot(X_train, w)
y_test_pred_custom = np.dot(X_test, w)

In [ ]:
# Other Linear Regression Metrics
print(f'Mean absolute error is:{mean_absolute_error(y_test, y_test_pred_custom): .2f}')
print(f'Root mean squared error is:{np.sqrt(mean_squared_error(y_test, y_test_pred_custom)): .2f}')

In [ ]:
r2_score(y_train, y_train_pred_custom)

In [ ]:
r2_score(y_test, y_test_pred_custom)

##### Plots

The following plots are made
1. Actual V/s Predicted - Plotting actual data against the fitted line - shows if there is clear tendency for points to be distributed around the line - in our case no.
2. KDE of actual and predicted - This shows whether probability density of predicted values ​​does approximates real values (or not) - In our case it does not
3. Residual versus predicted - This plot tells if the residuals are randomly distributed uniformly around the fitted line - in our case it is indeed the case, but the error is too high


In [ ]:
def plot_actual_vs_predicted(y, y_pred, title, x_label, y_label):
    #figure size
    plt.figure(figsize=(10, 7))
    
    #scatterplot of y y_pred
    plt.scatter(y, y_pred)
    plt.plot(y_test, y_test, color='r')

    #labeling
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)

    #showig plot
    plt.show()

In [ ]:
plot_actual_vs_predicted(y_test, y_test_pred_custom, 
                         'Actual versus predicted values for test data',
                         'Actual Values', 'Predicted values')

In [ ]:
def plot_kde_actual_vs_predicted(y, y_pred, title, actual_label, predicted_label):
    ''' 
    The plot(kde) function plots the KDE. Inputs are just real and predicted y values, in this order:
    y, y_pred
    '''
    #figsize
    plt.figure(figsize=(10, 7))

    #Kernel Density Estimation plot
    ax = sns.kdeplot(y, color='r', label=actual_label) #actual values
    sns.kdeplot(y_pred, color='b', label=predicted_label, ax=ax) #predicted values

    #showing title
    plt.title(title)
    #showing legend
    plt.legend()
    #showing plot
    plt.show()

In [ ]:
plot_kde_actual_vs_predicted(y_test, y_test_pred_custom, 
                             'Actual vs Predicted values', 'Actual Values', 'Predicted Values')

In [ ]:
def plot_residual_vs_predicted(y, y_pred, title, x_label, y_label):
    #figure size
    plt.figure(figsize=(10, 7))

    #residual plot
    sns.residplot(x=y, y=y_pred)

    #labeling
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)

In [ ]:
plot_residual_vs_predicted(y_test, y_test_pred_custom, 
                         'Residuals versus predicted values plot',
                         'Predicted values for Diabetes', 'Residuals')

Conclusion from plots:

1. Actual V/s Predicted Plot: Plotting actual data against the fitted line shows if there is NO clear tendency for points to be distributed around the line.
2. KDE of actual and predicted - Probability density of predicted values ​​does NOT approximate real values
3. Residual versus predicted: Residuals are indeed randomly distributed uniformly around the fitted line, but the error is too high. This confirms that linear regression is not a good model for this ML problem, and another one must be sought. A regularized version of Linear Regression will also not work because as we have seen, the R2 with training data is also very bad

We will next try out sklearn based
1. Linear Regression, followed by 
2. Ridge and Lasso Regression to demo 
    * Regularization by using GridSearchCV for tuning lambda hyperparameter and also 
    * To prove the point above wrt regularised Regression that it will not work
3. Finally a Polynomial Regression is also tried for completion

### 5. Use sklearn Linear Regression

In [ ]:
data = load_diabetes()

In [ ]:
X = data.data
y = data.target

# Convert the iris dataset to a pandas dataframe
df = pd.DataFrame(data.data, columns=["age","sex","bmi","bp", "tc", "ldl", "hdl","tch", "ltg", "glu"])

# Add the target variable to the dataframe
df['target'] = data.target

# Print the first 5 rows of the dataframe
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

R-squared is very low. This shows that very less of variance is explained by linear model and there is non linear component to it. 

In [ ]:
y_train_pred_sklearn = model.predict(X_train)
y_test_pred_sklearn = model.predict(X_test)

In [ ]:
# Other Linear Regression Metrics
print(f'Mean absolute error is:{mean_absolute_error(y_test, y_test_pred_custom): .2f}')
print(f'Root mean squared error is:{np.sqrt(mean_squared_error(y_test, y_test_pred_custom)): .2f}')

In [ ]:
r2_score(y_train, y_train_pred_sklearn)

In [ ]:
r2_score(y_test, y_test_pred_sklearn)

### 6. Lasso (L1) regularized Linear Regression

$$
\arg \min_w \nabla_w \mathcal{J} + \lambda_1 \nabla_w \|w\|_1 \\
\nabla_w \mathcal{J} = \frac{2}{m} X^T (Xw - y) \\
\nabla_w \|w\|_1 = \mathbf{1} \\
\textbf{w} = (\textbf{w} -\eta \lambda) - \eta \nabla_w \mathcal{J} 
$$

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

In [ ]:
#defining the lasso model
model = Lasso()

#define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# efine grid
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)

#define search - here using a mean absolute error instead of mean squared error
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

#performing the search on the train dataset
results = search.fit(X_train, y_train)

#printing
print(f'MAE:{results.best_score_: .2f}')
print(f'Best Alpha:{results.best_params_}')

In [ ]:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# define model
model = Lasso()

# define evaluation
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# define search space
space = dict()
space['alpha'] = loguniform(1e-5, 100)
space['fit_intercept'] = [True, False]

#define search
search = RandomizedSearchCV(model, space, n_iter=500, scoring='neg_mean_absolute_error', n_jobs=-1, cv=cv, random_state=1)

# execute search
result = search.fit(X, y)

#printing
print(f'MAE:{results.best_score_: .2f}')
print(f'Best Alpha:{results.best_params_}')

In [ ]:
#lasso with best alpha
model_best = Lasso(alpha=0.01).fit(X_train, y_train)

#predictions
y_train_pred = model_best.predict(X_train)
y_test_pred = model_best.predict(X_test)

print(r2_score(y_train, y_train_pred))
print(r2_score(y_test, y_test_pred))

The above metrics and alpha shows that regularization did not result in better test predictions. Further confirming that Linear model is not the way to go here

Exercise:
1. Try applying RidgeRegression

### 7. Polynomial Regression

We will use polynomial regression in a pipeline and perform grid search

In [ ]:
data = load_diabetes()

In [ ]:
X = data.data
y = data.target

# Convert the iris dataset to a pandas dataframe
df = pd.DataFrame(data.data, columns=["age","sex","bmi","bp", "tc", "ldl", "hdl","tch", "ltg", "glu"])

# Add the target variable to the dataframe
df['target'] = data.target

# Print the first 5 rows of the dataframe
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

def PolynomialRegression(degree=2):
    return Pipeline([('poly', PolynomialFeatures(degree)),
                     ('reg', LinearRegression())])

In [ ]:
model = PolynomialRegression(5)
model.fit(X_train, y_train)

In [ ]:
train_score = model.score(X_train, y_train)
print(f'R2 score (train): {train_score:.5f}')

test_score = model.score(X_test, y_test)
print(f'R2 score (test): {test_score:.5f}')

Observation from above result: Excellent R2 on train and really bad score on test indicates overfitting


##### 7.1 Visualizing polynomial fit efficacy

When the data set is not two-dimensional, it is not possible to plot the polynomials in order to find the best fit to the data. A plot that can help you find the optimal polynomial degree in this case is a validation curve. A validation curve is a graph that shows the training and validation scores for various values of a given parameter. It is similar to a grid search with a single parameter, but it also allows you to plot the results.


In [ ]:
from sklearn.model_selection import validation_curve

degree = np.arange(1, 6)
train_scores, val_scores = validation_curve(
                                PolynomialRegression(), 
                                X_train, y_train, 
                                param_name='poly__degree', 
                                param_range=degree, 
                                cv=10
                           )
plt.plot(degree, np.mean(train_scores, axis=1), 'b', label='training set')
plt.plot(degree, np.mean(val_scores, axis=1), 'r', label='validation set')
plt.legend()
plt.xlabel('degree')
plt.ylabel('$R^2$ score')

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'poly__degree': np.arange(1, 6),
}

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=10, n_jobs=-1)
grid.fit(X_train, y_train)

print(grid.best_params_)

Given that polynomial of degree 1 and 2 provide the same score, we stick to the simplest one - i.e. Linear Regression.

However it is interesting to see what Polynomial Features sklearn transformer does to the data X.
See next cell for reference

In [ ]:
poly = PolynomialFeatures(3)
poly.fit_transform(X_train) # Puts 1 in first cell, x, x^2 and x^3 feature crosses in subsequent cells

##### 7.1 Alternative approaches to polynomial regression 

Polynomial regression is one example of regression models that use basis functions to model the relationship between two variables. In this type of models, the target variable y is modeled as a linear combination of a set of d basis functions of the input x:

$$
h(x) = w_0 + w_1 \varphi_1(x) + w_2 \varphi_2(x) + ......
$$

In polynomial regression, the basis functions are just the powers of x. Other popular families of basis functions include radial basis functions (RBF), splines, and wavelets. These families often provide a better fit to the data than polynomials

### 8. Applying normal equation directly for small datasets
$$
w = (X^TX)^{-1}X^Ty 
$$

In [ ]:
data = load_diabetes()

In [ ]:
X = data.data
y = data.target

# Convert the iris dataset to a pandas dataframe
df = pd.DataFrame(data.data, columns=["age","sex","bmi","bp", "tc", "ldl", "hdl","tch", "ltg", "glu"])

# Add the target variable to the dataframe
df['target'] = data.target

# Print the first 5 rows of the dataframe
df.head()

In [ ]:
X_withones = np.hstack((np.ones(X.shape[0]).reshape(-1,1), X))
X_train, X_test, y_train, y_test = train_test_split(X_withones, y, test_size=0.2, random_state=42)

In [ ]:
# w = (X^TX)^-1 X^Ty
w = np.dot(
        np.dot(
            np.linalg.inv(np.dot(X_train.T, X_train)), 
            X_train.T), 
        y_train)
w

In [ ]:
y_test_pred = np.dot(X_test, w)

In [ ]:
r2_score(y_true=y_test, y_pred=y_test_pred)

### 9. OLS from Statsmodel package

In [ ]:
# https://www.kaggle.com/datasets/debajyotipodder/co2-emission-by-vehicles
df = pd.read_csv("data/co2_emissions.csv")
df.head()

In [ ]:
df.describe()

In [ ]:
# For ease of use in our regression, we drop character columns
# In reality you might want to encode these in some appropriate way

df.drop(['Make','Model','Vehicle Class','Transmission','Fuel Type'], axis = 1, inplace = True)

In [ ]:
from matplotlib import style

style.use('seaborn-whitegrid')
plt.rcParams['figure.figsize'] = (20,10)

In [ ]:
sns.pairplot(df)
#plt.savefig('pairplot.png')

In [ ]:
plt.scatter(x = 'Engine Size(L)', y = 'CO2 Emissions(g/km)', data = df, s = 100, alpha = 0.3, edgecolor = 'white')
plt.title('Engine size vs C02 Emissions', fontsize = 16)
plt.ylabel('CO2 Emissions', fontsize = 12)
plt.xlabel('Engine Size', fontsize = 12)

In [ ]:
plt.scatter(x = 'Fuel Consumption Comb (L/100 km)', y = 'CO2 Emissions(g/km)', data = df, s = 100, alpha = 0.3, edgecolor = 'white')
plt.title('Fuel Consumption Comb (L/100 km) vs C02 Emissions', fontsize = 16)
plt.ylabel('CO2 Emissions', fontsize = 12)
plt.xlabel('Fuel Consumption Comb (L/100 km)', fontsize = 12)

In [ ]:
sns.heatmap(df.corr(), annot = True, cmap = 'magma')

Observation: 
1. Fuel consumption is highly negatively correlated with every other feature.
2. A lot of other columns are highly positively correlated 

##### 9.1 Simple Linear Regression 

With one feature

In [ ]:
X_var = df[['Engine Size(L)']] # independent variable
y_var = df['CO2 Emissions(g/km)'] # dependent variable

1. !pip install statsmodels  - run this command in conda shell
2. !pip install termcolor

In [ ]:
import statsmodels.api as sm

slr_model = sm.OLS(y_var, X_var) # Ordinary Least Squares 
slr_reg = slr_model.fit()

In [ ]:
from termcolor import colored as cl

print(cl(slr_reg.summary(),attrs = ['bold']))

##### Multiple Linear Regression with OLS

In [ ]:
X1_var = df[['Engine Size(L)','Fuel Consumption Comb (L/100 km)',
             'Fuel Consumption Hwy (L/100 km)','Fuel Consumption City (L/100 km)']]
y_var = df['CO2 Emissions(g/km)'] # dependent variable


sm_X1_var = sm.add_constant(X1_var)
mlr_model = sm.OLS(y_var, sm_X1_var)
mlr_reg = mlr_model.fit()

In [ ]:
print(cl(mlr_reg.summary(), attrs = ['bold']))

Observation:
1. Not only Adjusted R-squared, but also R2 itself dropped when adding second feature

**Exercise**

Perform Linear Regression using the advertising dataset

### 10. Demoing usage of Lasso (L1) regression elimination of features

1. As alpha (lambda) increases, the number of features eliminated increases

Using the Energy efficiency data from https://archive.ics.uci.edu/dataset/242/energy+efficiency

**Important** !pip install openpyxl

In [ ]:
df = pd.read_excel("data/ENB2012_data.xlsx")
df.head()

1. X1 Relative Compactness
2. X2 Surface Area
3. X3 Wall Area
4. X4 Roof Area
5. X5 Overall Height
6. X6 Orientation
7. X7 Glazing Area
8. X8 Glazing Area Distribution
9. y1 Heating Load
10. y2 Cooling Load

In [ ]:
y1 = df.pop("Y1")
y2 = df.pop("Y2")
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=0.2, random_state=42)

In [ ]:
lasso_model = Lasso(alpha = 0.1)
lasso_model.fit(X=X_train, y=y_train)

In [ ]:
#extract the coefficients
df_coef_lasso = pd.DataFrame({"var": X_train.columns.values, 
                              "coef":lasso_model.coef_})
df_coef_lasso

In [ ]:
lasso_model.score(X_train, y_train)

In [ ]:
model_predictions = lasso_model.predict(X_test)
lasso_model.score(X=X_test, y=y_test)

In [ ]:
#import the necessary modules
from itertools import cycle
from sklearn.linear_model import lasso_path
import numpy as np
import matplotlib.pyplot as plt
#run the lasso path
alphas_lasso, coefs_lasso, _ = lasso_path(X_train, y_train.values.reshape(-1),
                                          alphas = [.0001, .001, .01,.1, .5, 1, 10, 100, 1000, 10000])
#plot the coefficients over the path
log_alphas_lasso = np.log10(alphas_lasso)
for index, coef_l in enumerate(coefs_lasso):
    l1 = plt.plot(log_alphas_lasso, coef_l,
                 label = X_train.columns.values[index])
#add labels
plt.xlabel('Log(alpha)')
plt.ylabel('coefficients')
plt.title('Lasso Path')
plt.axis('tight')
plt.legend(bbox_to_anchor = (0.7, 0.3, 0.5, 0.5))
#sho the model
plt.show()

**Observation**: The above plot shows how each feature becomes irrelevant as alpha increases

### 11. Heteroskedasticity


In [ ]:
df = pd.read_csv("data/moscow_apartment_listings.csv")
df.head()

In [ ]:
df.info()

Dropping categorical variables before OLS. To use categorical variables in OLS, refer to this - https://www.datarobot.com/blog/multiple-regression-using-statsmodels/. Alternatively you will also learn this in AAPS subject in second semester

In [ ]:
char_cols = df.select_dtypes(include=["object"]).columns.tolist()
print(char_cols)
df.drop(char_cols, axis=1, inplace=True)
df.head()

In [ ]:
#y = df.pop("price")
#X = df #.to_numpy()
#X = sm.add_constant(X)
#model = sm.OLS(y,X)
#model = model.fit()


In [ ]:
import statsmodels.formula.api as smf 
# formula: response ~ predictor + predictor 
est = smf.ols(formula='price ~ repair + year_built_empty + house_age + dist_to_subway', data=df).fit()

**Detecting Heteroscedasticity**
Among several dozen methods to detect Heteroskedasticity, here we use Het-White Test. The methodology is as follows:
1. First, we make two hypotheses: Null (H0) and Alternate (H1).
    * H0: The dataset has homoskedasticity.
    * H1: The dataset does not have homoskedasticity but Heteroscedasticity.
2. The test returns values for ‘Lagrange Multiplier statistic’, ‘LM test’s p-value’, ‘F-statistic’, and ‘F-test’s p-value’. 
3. If the P-value output is less than 0.05. Then we reject the null hypothesis

In [ ]:
from statsmodels.stats.diagnostic import het_white
from statsmodels.compat import lzip
from patsy import dmatrices

expr = 'price ~ repair + year_built_empty + house_age + dist_to_subway'
y, X = dmatrices(expr, df, return_type='dataframe')

keys = ['Lagrange Multiplier statistic:', 'LM test\'s p-value:', 'F-statistic:', 'F-test\'s p-value:']
results = het_white(model.resid, X)
lzip(keys, results)

### Multicollinearity

In [ ]:
df = pd.read_csv("data/weatherAUS.csv")
df.head()

In [ ]:
import statsmodels.api as sm              ## Performing statistical methods
from statsmodels.stats.outliers_influence import variance_inflation_factor    ## For checking Multicollinearity

Before calculating VIF, we need to remove columns with NA or impute

In [ ]:
df.info()

In [ ]:
# Missing percentage
(df.isna().sum(axis=0)/df.shape[0]).plot(kind = 'bar',ylim = (0,1))

In [ ]:
# Drop columns with more than 20% missing values
missing_percent = df.isna().sum(axis=0)/df.shape[0]
drop_cols = missing_percent[missing_percent > 0.2].index
df.drop(columns=drop_cols,inplace=True)
df.head()

In [ ]:
df.drop(columns = 'Date' , inplace=True)

In [ ]:
missing_percent = df.isna().sum(axis=0)/df.shape[0]
missing_percent.plot(kind='bar',ylim=(0,1))

In [ ]:
df.dropna(axis=0,inplace=True)

In [ ]:
df.info()

Deal with categorical columns

In [ ]:
df['RainTomorrow'].replace({'No':0, 'Yes': 1}, inplace = True)
df['RainToday'].replace({'No':0, 'Yes': 1}, inplace = True)

In [ ]:
numeric_cols = list(df.select_dtypes(exclude='object'))
categorical_cols = ['Location']
ordinal_cols = list(set(df.columns) - set(numeric_cols) - set(categorical_cols))
numeric_cols.remove('RainTomorrow')

In [ ]:
numeric_cols = list(df.select_dtypes(exclude='object'))
categorical_cols = ['Location']
ordinal_cols = list(set(df.columns) - set(numeric_cols) - set(categorical_cols))
numeric_cols.remove('RainTomorrow')

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
for feature in categorical_cols+ordinal_cols:
    label_encoder = LabelEncoder()
    df[feature] = label_encoder.fit_transform(df[feature])

In [ ]:
from scipy.stats import chi2_contingency
print("Relation with RainTomorrow at p = 0.05 ")
for feature in categorical_cols+ordinal_cols:
    data = pd.crosstab(df[feature],df['RainTomorrow'])
    stat, p , dof , expected = chi2_contingency(data)
    if p <= 0.05:
        print('{0} : Related ' .format(feature))
    else:
        print('{0} : Not Related' .format(feature))

In [ ]:
sns.set(rc = {'figure.figsize' : (16,16)})
sns.heatmap(df[numeric_cols].corr(), annot = True , 
            cmap=sns.color_palette("crest", as_cmap=True)
)

**Calculate VIF**

In [ ]:
vif_data = pd.DataFrame()
ind_features = df[numeric_cols]
vif_data['feature'] = ind_features.columns

In [ ]:
vif_data['VIF'] = [variance_inflation_factor(ind_features.values, i)
                       for i in range(len(ind_features.columns))]

In [ ]:
vif_data.sort_values(by='VIF',ascending=False)

**Observations**

1. A lots of very high VIFs...
2. But not all of these features need to be deleted
3. We can cleverly creagte some new features out of these as shown below


In [ ]:
# Make copy of current dataframe before changes
df_before_vif_fix = df.copy(deep = True)

In [ ]:
df['Pressure_Interval'] = abs(df['Pressure9am'] - df['Pressure3pm'])
df['Humidity_Interval'] = abs(df['Humidity9am'] - df['Humidity3pm'])
df['TempInterval'] = abs(df['Temp9am'] - df['Temp3pm'])
df['WindSpeedInterval'] = abs(df['WindSpeed9am'] - df['WindSpeed3pm'])
df['Temperature_Interval'] = abs(df['MaxTemp'] - df['MinTemp'])
df.head()

In [ ]:
#Drop the columns that contributed to the difference calc above
df.drop(columns=['Pressure9am','Pressure3pm','MaxTemp','MinTemp','WindDir9am','WindDir3pm',
                 'Humidity9am','Humidity3pm','Temp9am','Temp3pm','WindSpeed9am','WindSpeed3pm'],
                 axis=1,inplace=True)

In [ ]:
ind_features = df.drop('RainTomorrow',axis=1)

In [ ]:
ind_features.columns

In [ ]:
numeric_cols = list(df.select_dtypes(exclude='object'))
vif_data = pd.DataFrame()
ind_features = df[numeric_cols]
vif_data['feature'] = ind_features.columns

In [ ]:
vif_data["VIF"] = [variance_inflation_factor(ind_features.values, i)
                          for i in range(len(ind_features.columns))]

In [ ]:
vif_data.sort_values(by='VIF',ascending=False)

Drop features with VIF > 0 and apply linear regression

In [ ]:
df.drop(list(vif_data[vif_data['VIF']>5]['feature']),axis=1,inplace=True)

In [ ]:
df.head()